In [1]:
import pandas as pd
import numpy as np


In [2]:
campaigns = pd.read_csv('D:/datasets/assigment/campaigns.csv')
client_first_purchase_date = pd.read_csv('D:/datasets/assigment/client_first_purchase_date.csv')
events = pd.read_csv('D:/datasets/assigment/events.csv')
friends = pd.read_csv('D:/datasets/assigment/friends.csv')
messages = pd.read_csv('D:/datasets/assigment/messages.csv')

C:\Users\nikit\AppData\Local\Temp\ipykernel_14292\2938535284.py:5: DtypeWarning: Columns (7,8,16,17,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  messages = pd.read_csv('D:/datasets/assigment/messages.csv')


# Clean Data for PostgreSQL

In [4]:
def clean_campaigns(campaigns : pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    campaigns['total_count'] = campaigns['total_count'].astype('object')
    campaigns['hour_limit'] = campaigns['hour_limit'].astype('object')
    campaigns['subject_length'] = campaigns['subject_length'].astype('object')
    campaigns['position'] = campaigns['position'].astype('object')
    
    for i, row in campaigns.iterrows():
        if not pd.isna(row['total_count']):
            campaigns.at[i,'total_count'] = int(row['total_count'])
        if not pd.isna(row['hour_limit']):
            campaigns.at[i,'hour_limit'] = int(row['hour_limit'])
        if not pd.isna(row['subject_length']):
            campaigns.at[i,'subject_length'] = int(row['subject_length'])
        if not pd.isna(row['position']):
            campaigns.at[i,'position'] = int(row['position'])
        if pd.isna(row['ab_test']):
            campaigns.at[i,'ab_test'] = False
        # if pd.isna(row['hour_limit']):
        #     campaigns.at[i,'hour_limit'] = "null"
            
   
    
    return campaigns

In [5]:
def clean_events(events : pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    events['event_time'] = events['event_time'].astype(str).str.replace(" UTC", "", regex=False)
    events['event_time'] = pd.to_datetime(events['event_time'])
    
    events['price'] = pd.to_numeric(events['price'])
    
    return events

In [6]:
def clean_messages(messages : pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
      date_columns = [
        'date', 'sent_at', 'opened_first_time_at', 'opened_last_time_at',
        'clicked_first_time_at', 'clicked_last_time_at', 'unsubscribed_at',
        'hard_bounced_at', 'soft_bounced_at', 'complained_at', 'blocked_at',
        'purchased_at', 'created_at', 'updated_at'
      ]
      
      for col in date_columns:
          if col in messages.columns:
              messages[col] = pd.to_datetime(messages[col], errors='coerce')
              
      boolean_cols = [
        'is_opened', 'is_clicked', 'is_unsubscribed', 'is_hard_bounced',
        'is_soft_bounced', 'is_complained', 'is_blocked', 'is_purchased'
      ]
      
      for col in boolean_cols:
          if col in messages.columns:
              messages[col] = messages[col].map({'t': True, 'f': False})
              
      return messages

In [7]:
def clean_friends(friends : pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    friends['friend1'] = pd.to_numeric(friends['friend1'])
    friends['friend2'] = pd.to_numeric(friends['friend2'])
    
    return friends


In [8]:
def clean_client_first_purchase_date(cfp_data : pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    cfp_data['first_purchase_date'] = pd.to_datetime(cfp_data['first_purchase_date'])
    
    return cfp_data

In [9]:
clean_campaingns_data = clean_campaigns(campaigns)
clean_events_data = clean_events(events)
clean_messages_data = clean_messages(messages)
clean_friends_data = clean_friends(friends)
clean_client_first_purchase_date_data = clean_client_first_purchase_date(client_first_purchase_date)

In [10]:
clean_campaingns_data.to_csv('D:/datasets/assigment/cleaned/Clean_Campaings.csv', index=False)
clean_events_data.to_csv('D:/datasets/assigment/cleaned/Clean_Events.csv', index=False)
clean_messages_data.to_csv('D:/datasets/assigment/cleaned/Clean_Messages.csv', index=False)
clean_friends_data.to_csv('D:/datasets/assigment/cleaned/Clean_Friends.csv', index=False)
client_first_purchase_date.to_csv('D:/datasets/assigment/cleaned/Clean_client_first_purchase_date.csv', index=False)

# Clean Data for MongoDB

In [3]:
import json
from datetime import datetime

In [4]:
def processing_nan(value_in_row):
     if not pd.isna(value_in_row):
            return int(value_in_row)
     else:
         return None

In [5]:
def processing_date(dates_in_row):
    return pd.to_datetime(dates_in_row).isoformat()

In [6]:
def preproc_campaings(row):


    
    composite_key = str(row['id']) + '_' + row['campaign_type']
    
    mongo_row = {
        '_id': composite_key,
        'id': row['id'],
        'campaign_type': row['campaign_type'],
        'channel': row['channel'],
        'topic': row['topic'],
        'started_at': processing_date(row['started_at']),
        'finished_at': processing_date(row['finished_at']),
        'total_count': processing_nan(row['total_count']),
        'ab_test': row['ab_test'],
        'warmup_mode': row['warmup_mode'],
        'subject' : {
            'length' : processing_nan(row['subject_length']),
            'with_personalization' : row['subject_with_personalization'],
            'with_deadline' : row['subject_with_deadline'],
            'with_emoji': row['subject_with_emoji'],
            'with_bonuses': row['subject_with_bonuses'],
            'with_discount' : row['subject_with_discount'],
            'with_saleout' : row['subject_with_saleout'],
        },
        'is_test': row['is_test'],
        'position': processing_nan(row['position']),
    }    

        
        
    return mongo_row
    

In [29]:
def preproc_event(row):



    mongo_row = {
        'event_time': processing_date(row['event_time']),
        'event_type': row['event_type'],
        'product':{
            'product_id': int(row['product_id']),
            'category_id': int(row['category_id']),
            'category_code': row['category_code'],
            'brand': row['brand'],
            'price': processing_nan(row['price']),
        },
        'user':{
            'user_id': int(row['user_id']),
            'user_session': row['user_session'],
        }
    }

        

    return mongo_row  

In [30]:
def preproc_friends(row):
    

    mongo_row = {
        'friend1' : int(row['friend1']),
        'friend2' : int(row['friend2']),
    }
    

        
    return mongo_row
        

In [31]:
def preproc_client_first_purchase_date(row):

    
    mongo_row = {
        'client_id' : int(row['client_id']),
        'first_purchase_date' : processing_date(row['first_purchase_date']),
        'user_id' : int(row['user_id']),
        'user_device_id': int(row['user_device_id']),
    }
    
   
        
    
    return mongo_row

In [9]:

def preproc_messages(row):
        
    mongo_row = {
        'message_id': row['message_id'],
        'campaign_id' : int(row['campaign_id']),
        'message_type': row['message_type'],
        'client_id' : int(row['client_id']),
        'channel' : row['channel'],
        'platform' : row['platform'],
        'email_provider' : row['email_provider'],
        'stream' : row['stream'],
        'sent_at': processing_date(row['sent_at']),
        'status':{
            'is_opened' : row['is_opened'],
            'opened_at' : {
                'first' : processing_date(row['opened_first_time_at']),
                'last' : processing_date(row['opened_last_time_at']),
            },
            'is_clicked': row['is_clicked'],
            'clicked_at' : {
                'first' : processing_date(row['clicked_first_time_at']),
                'last' : processing_date(row['clicked_last_time_at']),
            },
            'is_unsubscribed': row['is_unsubscribed'],
            'is_hard_bounced': row['is_hard_bounced'],
            'is_soft_bounced': row['is_soft_bounced'],
            'is_complained' : row['is_complained'],
            'is_blocked': row['is_blocked'],
            'is_purchased': row['is_purchased'],
            'purchased_at' : processing_date(row['purchased_at']),
        },
        'created_at': processing_date(row['created_at']),
        'updated_at': processing_date(row['updated_at']),
        'user_device_id' : int(row['user_device_id']),
        'user_id': int(row['user_id']),
        
    }

    return mongo_row

In [7]:
campaigns_json_preproc = campaigns.apply(preproc_campaings, axis=1)


In [40]:
events_json_preproc = events.apply(preproc_event, axis=1)

In [41]:
friends_json_preproc = friends.apply(preproc_friends, axis=1)

In [42]:
client_first_purchase_date_json_preproc = client_first_purchase_date.apply(preproc_client_first_purchase_date, axis=1)


In [ ]:
messages_json_preproc = messages.apply(preproc_messages, axis=1)


In [ ]:
path = 'D:/datasets/assigment/cleaned/MongoDB_Data/'

In [55]:
campaigns_json_preproc.reset_index().to_json(path+'campaigns.json', orient='records', date_format='iso')

In [56]:
events_json_preproc.reset_index().to_json(path+'events.json', orient='records', date_format='iso')

In [57]:
friends_json_preproc.reset_index().to_json(path+'friends.json', orient='records', date_format='iso')

In [58]:
client_first_purchase_date_json_preproc.reset_index().to_json(path+'cfp_date.json', orient='records', date_format='iso')

In [8]:
campaigns_json_preproc

0       {'_id': '63_bulk', 'id': 63, 'campaign_type': ...
1       {'_id': '64_bulk', 'id': 64, 'campaign_type': ...
2       {'_id': '78_bulk', 'id': 78, 'campaign_type': ...
3       {'_id': '79_bulk', 'id': 79, 'campaign_type': ...
4       {'_id': '89_bulk', 'id': 89, 'campaign_type': ...
                              ...                        
1902    {'_id': '179_transactional', 'id': 179, 'campa...
1903    {'_id': '35_transactional', 'id': 35, 'campaig...
1904    {'_id': '57_transactional', 'id': 57, 'campaig...
1905    {'_id': '56_transactional', 'id': 56, 'campaig...
1906    {'_id': '237_transactional', 'id': 237, 'campa...
Length: 1907, dtype: object

In [59]:
del(campaigns_json_preproc)
del(events_json_preproc)
del(friends_json_preproc)
del(client_first_purchase_date_json_preproc)

### Clean Data for Neo4j


In [1]:
import pandas as pd
from datetime import datetime

In [10]:


def process_data_neo():

    campaigns = pd.read_csv('.venv/BigData/data/campaigns.csv', parse_dates=['started_at', 'finished_at'])
    campaigns = campaigns.drop_duplicates('id')
    

    events = pd.read_csv('.venv/BigData/data/events.csv', parse_dates=['event_time'])
    events['event_time'] = events['event_time'].dt.tz_localize(None)
    

    messages = pd.read_csv('.venv/BigData/data/messages.csv')
    bool_columns = ['is_opened', 'is_clicked', 'is_unsubscribed']
    messages[bool_columns] = messages[bool_columns].replace({'t': True, 'f': False})
    messages['sent_at'] = pd.to_datetime(messages['sent_at'])
    
  
    friends = pd.read_csv('.venv/BigData/data/friends.csv')
    

    clients = pd.read_csv('.venv/BigData/data/client_first_purchase_date.csv', parse_dates=['first_purchase_date'])
    

    campaigns.to_csv('.venv/BigData/data/cleaned_data_neo/processed_campaigns.csv', index=False)
    events.to_csv('.venv/BigData/data/cleaned_data_neo/processed_events.csv', index=False)
    messages.to_csv('.venv/BigData/data/cleaned_data_neo/processed_messages.csv', index=False)
    friends.to_csv('.venv/BigData/data/cleaned_data_neo/processed_friends.csv', index=False)
    clients.to_csv('.venv/BigData/data/cleaned_data_neo/processed_clients.csv', index=False)

In [11]:
process_data_neo()

C:\Users\nikit\AppData\Local\Temp\ipykernel_15780\2990645110.py:11: DtypeWarning: Columns (7,8,16,17,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  messages = pd.read_csv('.venv/BigData/data/messages.csv')
C:\Users\nikit\AppData\Local\Temp\ipykernel_15780\2990645110.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  messages[bool_columns] = messages[bool_columns].replace({'t': True, 'f': False})
